In [ ]:
import bibliograph as bg

bibtex_fname = "bibliograph/test_data/bibtex_test_data_short.bib"
entry_syntax_fname = "bibliograph/resources/default_bibtex_syntax.csv"

tn = bg.slurp_bibtex(
    bibtex_fname,
    entry_syntax_fname,
    syntax_case_sensitive=False,
    space_char='|',
    na_string_values='!',
    na_node_type='missing'
)

tn.strings

In [ ]:
import bibliograph as bg

shorthand_fname='bibliograph/test_data/manual_annotation.shnd'
entry_syntax_fname = "bibliograph/resources/default_entry_syntax.csv"
link_syntax_fname="bibliograph/resources/default_link_syntax.csv"

tn = bg.slurp_shorthand(
    shorthand_fname,
    entry_syntax_fname,
    link_syntax_fname,
    syntax_case_sensitive=False,
    item_separator='__',
    space_char='|',
    na_string_values='!',
    na_node_type='missing',
    default_entry_prefix='wrk',
    skiprows=2,
    comment_char='#'
)

In [6]:
import bibliograph as bg

shorthand_fname='bibliograph/test_data/shorthand_with_aliases.shnd'
entry_syntax_fname = "bibliograph/resources/default_entry_syntax.csv"
link_syntax_fname="bibliograph/resources/default_link_syntax.csv"

aliases_dict = {
    'actor': 'bibliograph/test_data/aliases_actor.csv',
    'work': 'bibliograph/test_data/aliases_work.csv'
}

tn = bg.slurp_shorthand(
    shorthand_fname,
    entry_syntax_fname,
    link_syntax_fname,
    syntax_case_sensitive=False,
    aliases_dict=aliases_dict,
    aliases_case_sensitive=False,
    item_separator='__',
    space_char='|',
    na_string_values='!',
    na_node_type='missing',
    default_entry_prefix='wrk',
    skiprows=2,
    comment_char='#',
)

tn.resolve_nodes().query('node_type == "actor"')

,node_type,name_string,abbr_string,date_inserted,date_modified
0,actor,alice smith,asmith,2022-07-22 15:22:44,<NA>
1,actor,elizabeth wu,bwu,2022-07-22 15:22:44,<NA>
2,actor,national aeronautics and space administration,nasa,2022-07-22 15:22:44,<NA>
19,actor,University of Chicago,University of Chicago,2022-07-22 15:22:44,<NA>


# TODO: alias insertion executes without error, now write test cases

In [5]:
!pytest bibliograph/tests.py

In [ ]:
import pandas as pd
strings = pd.DataFrame({'string':['a','A','b','c','d','C','z']})
data = pd.Series([1,1,2,4,5,6,3,3,2])

data = pd.concat([data, data.map(strings.string)], axis='columns')
data[2] = data[1].str.casefold()
folded_uid = data[2].unique()
folded_uid = pd.Series(range(len(folded_uid)), index=folded_uid)
data[3] = data[2].map(folded_uid)
data

In [ ]:
pd.